In [7]:
import psycopg2
import pandas as pd

In [8]:
# Create function to create database

def create_database():
    # Connect to database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=bigbywolf ")
    # Use connection to get cursor to execute queries
    cur = conn.cursor()
    # Set automatic commit
    conn.set_session(autocommit=True)
    
    # Create database
    cur.execute("DROP DATABASE IF EXISTS dog_adoption")
    cur.execute("CREATE DATABASE dog_adoption")
    
    # Close connection to default database
    conn.close()
    
    # Connect to newly created database
    conn = psycopg2.connect("host=127.0.0.1 dbname=dog_adoption user=bigbywolf")
    # Use connection to get cursor to execute queries
    cur = conn.cursor()
    # Set automatic commit
    conn.set_session(autocommit=True)
    
    return cur, conn

In [9]:
description = pd.read_csv("/Users/bigbywolf/Dog_Project/Dog_Project/Datasets/description.csv", low_memory=False)
description.dropna(subset=['index', 'id'], inplace=True)
description['index'] = description['index'].astype(int)
description['id'] = description['id'].astype(int)
description.head()

,index,id,org_id,url,breed_primary,breed_secondary,color_primary,color_secondary,color_tertiary,age,...,size,coat,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ
0,0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,American Staffordshire Terrier,Mixed Breed,White / Cream,Yellow / Tan / Blond / Fawn,NaN,Senior,...,Medium,Short,HARLEY,adoptable,2019-09-20T16:37:59+0000,Las Vegas,NV,89147.0,US,89009
1,1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Pit Bull Terrier,Mixed Breed,Brown / Chocolate,White / Cream,NaN,Adult,...,Large,Short,BIGGIE,adoptable,2019-09-20T16:24:57+0000,Las Vegas,NV,89147.0,US,89009
2,2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Shepherd,NaN,Brindle,NaN,NaN,Adult,...,Large,Short,Ziggy,adoptable,2019-09-20T14:10:11+0000,Mesquite,NV,89027.0,US,89009
3,3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,German Shepherd Dog,NaN,NaN,NaN,NaN,Baby,...,Large,NaN,Gypsy,adoptable,2019-09-20T10:08:22+0000,Pahrump,NV,89048.0,US,89009
4,4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dachshund,NaN,NaN,NaN,NaN,Young,...,Small,Long,Theo,adoptable,2019-09-20T06:48:30+0000,Henderson,NV,89052.0,US,89009


In [10]:
travel = pd.read_csv("/Users/bigbywolf/Dog_Project/Dog_Project/Datasets/travel.csv")
travel.head()

,index,id,contact_city,contact_state,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Afghanistan,NaN,NaN,NaN


In [11]:
location = pd.read_csv("/Users/bigbywolf/Dog_Project/Dog_Project/Datasets/location.csv")
location.head()

,index,location,exported,imported,total,inUS
0,0,Texas,635.0,NaN,566.0,True
1,1,Alabama,268.0,2.0,1428.0,True
2,2,North Carolina,158.0,14.0,2627.0,True
3,3,South Carolina,139.0,12.0,1618.0,True
4,4,Georgia,137.0,19.0,3479.0,True


In [12]:
cur, conn = create_database()

In [13]:
create_description_table = ("""CREATE TABLE IF NOT EXISTS description \
                (index INT PRIMARY KEY, id INT, org_id VARCHAR, url VARCHAR, breed_primary VARCHAR, \
                breed_secondary VARCHAR, color_primary VARCHAR, color_secondary VARCHAR, color_tertiary VARCHAR, \
                age VARCHAR, sex VARCHAR, size VARCHAR, coat VARCHAR, name VARCHAR, status VARCHAR, \
                posted VARCHAR, contact_city VARCHAR, contact_state VARCHAR, contact_zip VARCHAR, \
                contact_country VARCHAR, stateQ VARCHAR)""")
cur.execute(create_description_table)
conn.commit()

In [14]:
create_travel_table = ("""CREATE TABLE IF NOT EXISTS travel \
                (index INT PRIMARY KEY, id INT, contact_city VARCHAR, contact_state VARCHAR, \
                found VARCHAR, manual VARCHAR, remove VARCHAR, still_there VARCHAR)""")

cur.execute(create_travel_table)
conn.commit()

In [15]:
create_location_table = ("""CREATE TABLE IF NOT EXISTS location \
                (index INT PRIMARY KEY, location VARCHAR, exported FLOAT, imported FLOAT, \
                total FLOAT, inUS BOOL)""")

cur.execute(create_location_table)
conn.commit()

In [16]:
# Due to primary key constraint, before we insert data, we need to make sure there is no data that will cause duplicate
truncate_command = "TRUNCATE TABLE description;"
cur.execute(truncate_command)
insert_description_table = ("""INSERT INTO description \
                (index, id, org_id, url, breed_primary, breed_secondary, \
                color_primary, color_secondary, color_tertiary, \
                age, sex, size, coat, name, status, \
                posted, contact_city, contact_state, contact_zip, \
                contact_country, stateQ)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""")


# Iterating through index (i) and rows in description table
for i, row in description.iterrows():
    # Pass INSERT INTO query in cur.execute() and list of data
    cur.execute(insert_description_table, list(row))
conn.commit()

In [17]:
# Due to primary key constraint, before we insert data, we need to make sure there is no data that will cause duplicate
truncate_command = "TRUNCATE TABLE travel;"
cur.execute(truncate_command)
insert_travel_table = ("""INSERT INTO travel \
                (index, id, contact_city, contact_state, \
                found, manual, remove, still_there)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""")

for i, row in travel.iterrows(): # Iterating row by row in description table
    cur.execute(insert_travel_table, list(row))
conn.commit()

In [18]:
# Due to primary key constraint, before we insert data, we need to make sure there is no data that will cause duplicate
truncate_command = "TRUNCATE TABLE location;"
cur.execute(truncate_command)
insert_location_table = ("""INSERT INTO location \
                (index, location, exported, imported, \
                total, inUS)
                VALUES (%s, %s, %s, %s, %s, %s)""")

for i, row in location.iterrows(): # Iterating row by row in description table
    cur.execute(insert_location_table, list(row))
conn.commit()

In [20]:
if cur:
        cur.close()
if conn:
        conn.close()